# Tâche de classification de texte

Comme nous l'avons mentionné, nous allons nous concentrer sur une tâche simple de classification de texte basée sur le dataset **AG_NEWS**, qui consiste à classer les titres d'actualités dans l'une des 4 catégories : Monde, Sports, Économie et Sci/Tech.

## Le Dataset

Ce dataset est intégré dans le module [`torchtext`](https://github.com/pytorch/text), ce qui nous permet d'y accéder facilement.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Ici, `train_dataset` et `test_dataset` contiennent des collections qui renvoient respectivement des paires d'étiquette (numéro de classe) et de texte, par exemple :


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Alors, imprimons les 10 premiers nouveaux titres de notre ensemble de données :


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Parce que les ensembles de données sont des itérateurs, si nous voulons utiliser les données plusieurs fois, nous devons les convertir en liste :


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenisation

Nous devons maintenant convertir le texte en **nombres** pouvant être représentés sous forme de tenseurs. Si nous souhaitons une représentation au niveau des mots, nous devons effectuer deux étapes :
* utiliser un **tokeniseur** pour diviser le texte en **tokens**
* construire un **vocabulaire** à partir de ces tokens.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

En utilisant le vocabulaire, nous pouvons facilement encoder notre chaîne tokenisée en un ensemble de nombres :


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Représentation textuelle par sac de mots

Parce que les mots véhiculent du sens, il est parfois possible de comprendre le sens d'un texte simplement en regardant les mots individuels, indépendamment de leur ordre dans la phrase. Par exemple, pour classifier des articles de presse, des mots comme *météo*, *neige* sont susceptibles d'indiquer une *prévision météorologique*, tandis que des mots comme *actions*, *dollar* pourraient correspondre à des *nouvelles financières*.

La représentation vectorielle **Sac de mots** (BoW) est la méthode traditionnelle la plus couramment utilisée. Chaque mot est associé à un indice de vecteur, et l'élément du vecteur contient le nombre d'occurrences d'un mot dans un document donné.

![Image montrant comment une représentation vectorielle par sac de mots est stockée en mémoire.](../../../../../lessons/5-NLP/13-TextRep/images/bag-of-words-example.png) 

> **Note** : Vous pouvez également considérer BoW comme la somme de tous les vecteurs encodés en one-hot pour les mots individuels du texte.

Voici un exemple de génération d'une représentation par sac de mots en utilisant la bibliothèque Python Scikit Learn :


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Pour calculer le vecteur sac-de-mots à partir de la représentation vectorielle de notre ensemble de données AG_NEWS, nous pouvons utiliser la fonction suivante :


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Remarque :** Ici, nous utilisons la variable globale `vocab_size` pour spécifier la taille par défaut du vocabulaire. Étant donné que la taille du vocabulaire est souvent assez grande, nous pouvons limiter la taille du vocabulaire aux mots les plus fréquents. Essayez de réduire la valeur de `vocab_size` et d'exécuter le code ci-dessous, et observez comment cela affecte la précision. Vous devriez vous attendre à une légère baisse de précision, mais pas dramatique, en échange d'une meilleure performance.


## Entraîner un classificateur BoW

Maintenant que nous avons appris à construire une représentation Bag-of-Words pour notre texte, entraînons un classificateur par-dessus. Tout d'abord, nous devons convertir notre ensemble de données pour l'entraînement de manière à ce que toutes les représentations vectorielles positionnelles soient transformées en représentation Bag-of-Words. Cela peut être réalisé en passant la fonction `bowify` comme paramètre `collate_fn` au `DataLoader` standard de torch :


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Définissons maintenant un réseau de neurones classificateur simple qui contient une couche linéaire. La taille du vecteur d'entrée est égale à `vocab_size`, et la taille de sortie correspond au nombre de classes (4). Étant donné que nous résolvons une tâche de classification, la fonction d'activation finale est `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Maintenant, nous allons définir une boucle d'entraînement standard avec PyTorch. Étant donné que notre ensemble de données est assez volumineux, pour notre objectif pédagogique, nous n'entraînerons que pendant une seule époque, et parfois même moins d'une époque (la spécification du paramètre `epoch_size` nous permet de limiter l'entraînement). Nous rapporterons également l'exactitude accumulée de l'entraînement pendant la formation ; la fréquence de rapport est spécifiée à l'aide du paramètre `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrams, TriGrams et N-Grams

Une limitation de l'approche par sac de mots est que certains mots font partie d'expressions composées de plusieurs mots. Par exemple, le mot 'hot dog' a une signification complètement différente des mots 'hot' et 'dog' dans d'autres contextes. Si nous représentons toujours les mots 'hot' et 'dog' par les mêmes vecteurs, cela peut perturber notre modèle.

Pour résoudre ce problème, les **représentations N-gram** sont souvent utilisées dans les méthodes de classification de documents, où la fréquence de chaque mot, bi-mot ou tri-mot constitue une caractéristique utile pour entraîner des classificateurs. Dans une représentation bigramme, par exemple, nous ajouterons toutes les paires de mots au vocabulaire, en plus des mots originaux.

Voici un exemple de génération d'une représentation par sac de mots bigramme en utilisant Scikit Learn :


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Le principal inconvénient de l'approche N-gram est que la taille du vocabulaire commence à croître de manière extrêmement rapide. En pratique, il est nécessaire de combiner la représentation N-gram avec certaines techniques de réduction de dimensionnalité, comme les *embeddings*, que nous aborderons dans la prochaine unité.

Pour utiliser la représentation N-gram dans notre jeu de données **AG News**, nous devons construire un vocabulaire spécifique aux n-grams :


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Nous pourrions alors utiliser le même code que ci-dessus pour entraîner le classificateur, cependant, cela serait très inefficace en termes de mémoire. Dans la prochaine unité, nous entraînerons un classificateur bigramme en utilisant des embeddings.

> **Note:** Vous pouvez conserver uniquement les ngrams qui apparaissent dans le texte plus souvent qu'un nombre spécifié de fois. Cela garantira que les bigrammes peu fréquents seront omis et réduira considérablement la dimensionnalité. Pour ce faire, définissez le paramètre `min_freq` à une valeur plus élevée et observez le changement de longueur du vocabulaire.


## Fréquence Terme-Fréquence Inverse de Document TF-IDF

Dans la représentation BoW, les occurrences des mots sont pondérées de manière égale, quel que soit le mot lui-même. Cependant, il est évident que les mots fréquents, tels que *un*, *dans*, etc., sont beaucoup moins importants pour la classification que les termes spécialisés. En réalité, dans la plupart des tâches de NLP, certains mots sont plus pertinents que d'autres.

**TF-IDF** signifie **fréquence terme–fréquence inverse de document**. C'est une variation du sac de mots, où au lieu d'une valeur binaire 0/1 indiquant la présence d'un mot dans un document, une valeur en virgule flottante est utilisée, qui est liée à la fréquence d'apparition du mot dans le corpus.

Plus formellement, le poids $w_{ij}$ d'un mot $i$ dans le document $j$ est défini comme suit :
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
où
* $tf_{ij}$ est le nombre d'occurrences de $i$ dans $j$, c'est-à-dire la valeur BoW que nous avons vue précédemment
* $N$ est le nombre de documents dans la collection
* $df_i$ est le nombre de documents contenant le mot $i$ dans l'ensemble de la collection

La valeur TF-IDF $w_{ij}$ augmente proportionnellement au nombre de fois qu'un mot apparaît dans un document et est ajustée par le nombre de documents du corpus contenant ce mot, ce qui permet de corriger le fait que certains mots apparaissent plus fréquemment que d'autres. Par exemple, si le mot apparaît dans *chaque* document de la collection, $df_i=N$, et $w_{ij}=0$, ces termes seraient alors complètement ignorés.

Vous pouvez facilement créer une vectorisation TF-IDF de texte en utilisant Scikit Learn :


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Conclusion 

Cependant, bien que les représentations TF-IDF attribuent un poids de fréquence à différents mots, elles ne parviennent pas à représenter le sens ou l'ordre. Comme l'a dit le célèbre linguiste J. R. Firth en 1935 : « Le sens complet d'un mot est toujours contextuel, et aucune étude du sens en dehors du contexte ne peut être prise au sérieux. ». Nous apprendrons plus tard dans le cours comment capturer les informations contextuelles à partir du texte en utilisant la modélisation du langage.



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
